In [1]:
import pyLDAvis.gensim
import gensim
from gensim import corpora
from main import _get_data_from_cmd
from main import _parse_files
from main import _get_data_distribution
import pickle
from gensim import models

c:\program files\python35\lib\site-packages\_pytest\fixtures.py:834: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
c:\program files\python35\lib\site-packages\_pytest\fixtures.py:836: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)
c:\program files\python35\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\program files\python35\lib\site-packages\scipy\sparse\sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Загружаем словари текстов (в любом случае) и функцию для сортировки

In [2]:
 with open('new_output/pos_lemmas_with_normalized_words.pickle', 'rb') as handle:
        all_texts_dict_pos = pickle.load(handle)

In [3]:
 with open('new_output/raw_lemmas_with_normalized_words.pickle', 'rb') as handle:
        all_texts_dict_raw = pickle.load(handle)

In [4]:
def byName_key(topic):
    return topic[1]

Если что-то хочется (какой-то год) запустить отельно, делаем следущее:
* выбираем частотность (0 или 20)
* выбираем количество топиков (10 или 50)
* выбираем теги (0 - есть теги по частям речи, 1 - нет тегов по частям речи)
* выбираем год

In [5]:
frq = 0
topics = 10
raw = 0
year = 2015

if raw == 1:
    data = all_texts_dict_raw
elif raw == 0:
    data = all_texts_dict_pos
    
data = _get_data_distribution(data, "years/"+str(year)+"_en.pickle")

In [6]:
ldamodel = gensim.models.ldamodel.LdaModel.load("new_output/years/lda_model_normalized_words_freq_"+str(year)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+"/lda_model_normalized_words_freq_"+str(year)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(year)+"_en.mdl")
print("/lda_model_normalized_words_freq_"+str(year)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(year)+"_en.mdl")

/lda_model_normalized_words_freq_2015_en0_topics_10_pos_02015_en.mdl


In [7]:
top_topics = {}
dictionary = corpora.Dictionary(data)
dictionary.filter_n_most_frequent(frq)
corpus = [dictionary.doc2bow(doc) for doc in data]

In [8]:
ldamodel.log_perplexity(corpus)

-8.257717945936653

In [9]:
visdata = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [10]:
pyLDAvis.display(visdata)

In [11]:
vals = list(all_texts_dict_pos.values())
names = list(all_texts_dict_pos.keys())

all_topics = dict(ldamodel.print_topics(num_topics=topics, num_words=5))
for a in all_topics:
    print(a,": ",  all_topics[a])
print()

#names[i] -- сюда можно вставить айдишник текста и проверить на правильность
for i in range(5):
    print(list(reversed(sorted(ldamodel.get_document_topics(dictionary.doc2bow(vals[i])), key=byName_key)))[:3], names[i])

0 :  0.013*"word_NOUN" + 0.006*"dictionary_NOUN" + 0.006*"language_NOUN" + 0.006*"text_NOUN" + 0.005*"error_NOUN"
1 :  0.000*"text_NOUN" + 0.000*"relation_NOUN" + 0.000*"word_NOUN" + 0.000*"Russian_ADJ" + 0.000*"language_NOUN"
2 :  0.007*"feature_NOUN" + 0.006*"node_NOUN" + 0.006*"system_NOUN" + 0.006*"algorithm_NOUN" + 0.006*"datum_NOUN"
3 :  0.009*"text_NOUN" + 0.007*"word_NOUN" + 0.007*"language_NOUN" + 0.006*"example_NOUN" + 0.006*"relation_NOUN"
4 :  0.013*"word_NOUN" + 0.012*"text_NOUN" + 0.011*"term_NOUN" + 0.006*"system_NOUN" + 0.005*"datum_NOUN"
5 :  0.008*"word_NOUN" + 0.008*"term_NOUN" + 0.007*"result_NOUN" + 0.007*"task_NOUN" + 0.007*"sentiment_NOUN"
6 :  0.012*"construction_NOUN" + 0.012*"role_NOUN" + 0.011*"semantic_ADJ" + 0.010*"verb_NOUN" + 0.006*"FrameBank_PROPN"
7 :  0.011*"word_NOUN" + 0.009*"model_NOUN" + 0.009*"text_NOUN" + 0.009*"semantic_ADJ" + 0.008*"feature_NOUN"
8 :  0.009*"word_NOUN" + 0.009*"verb_NOUN" + 0.007*"language_NOUN" + 0.006*"Russian_ADJ" + 0.006*"t

## Если хочется просто все запустить

In [15]:
years = list(range(2001, 2018))
del years[1]

#### По всем годам с частями речи, topics = 10, removed 20 most freq_words

In [19]:
frq = 20
topics = 10
raw = 0
if raw == 1:
    data = all_texts_dict_raw
elif raw == 0:
    data = all_texts_dict_pos
else:
    raise Exception
old_data = data
for y in years:
    data = _get_data_distribution(old_data, "years/"+str(y)+"_en.pickle")
    ldamodel = gensim.models.ldamodel.LdaModel.load("new_output/years/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+"/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(y)+"_en.mdl")
    print("/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(y)+"_en.mdl")
    top_topics = {}
    dictionary = corpora.Dictionary(data)
    dictionary.filter_n_most_frequent(frq)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    print("log =",ldamodel.log_perplexity(corpus))
    all_topics = dict(ldamodel.print_topics(num_topics=topics, num_words=5))
    for a in all_topics:
        print("topic №"+str(a),": ",  " kwds ", all_topics[a])
    print()
    print()
    print()
    print("*"*20)
    
    

/lda_model_normalized_words_freq_2001_en20_topics_10_pos_02001_en.mdl
log = -7.29051526544
topic №0 :   kwds  0.001*"recognition_NOUN" + 0.001*"answer_NOUN" + 0.001*"version_NOUN" + 0.001*"datum_NOUN" + 0.001*"verification_NOUN"
topic №1 :   kwds  0.001*"recognition_NOUN" + 0.001*"speech_NOUN" + 0.001*"version_NOUN" + 0.001*"datum_NOUN" + 0.001*"performance_NOUN"
topic №2 :   kwds  0.043*"recognition_NOUN" + 0.024*"verification_NOUN" + 0.024*"version_NOUN" + 0.023*"datum_NOUN" + 0.017*"line_NOUN"
topic №3 :   kwds  0.024*"speech_NOUN" + 0.018*"consonant_NOUN" + 0.016*"diphone_NOUN" + 0.014*"error_NOUN" + 0.012*"fifth_ADJ"
topic №4 :   kwds  0.001*"recognition_NOUN" + 0.001*"speech_NOUN" + 0.001*"answer_NOUN" + 0.001*"question_NOUN" + 0.001*"possible_ADJ"
topic №5 :   kwds  0.001*"recognition_NOUN" + 0.001*"version_NOUN" + 0.001*"verification_NOUN" + 0.001*"datum_NOUN" + 0.001*"performance_NOUN"
topic №6 :   kwds  0.001*"recognition_NOUN" + 0.001*"version_NOUN" + 0.001*"verification_NOU

log = -8.01900428927
topic №0 :   kwds  0.000*"language_NOUN" + 0.000*"event_NOUN" + 0.000*"role_NOUN" + 0.000*"image_NOUN" + 0.000*"ller_NOUN"
topic №1 :   kwds  0.000*"speech_NOUN" + 0.000*"event_NOUN" + 0.000*"role_NOUN" + 0.000*"ller_NOUN" + 0.000*"based_VERB"
topic №2 :   kwds  0.019*"event_NOUN" + 0.012*"dictionary_NOUN" + 0.012*"ller_NOUN" + 0.011*"role_NOUN" + 0.009*"system_NOUN"
topic №3 :   kwds  0.000*"speech_NOUN" + 0.000*"word_NOUN" + 0.000*"Estonian_ADJ" + 0.000*"corpus_ADJ" + 0.000*"language_NOUN"
topic №4 :   kwds  0.000*"type_NOUN" + 0.000*"language_NOUN" + 0.000*"image_NOUN" + 0.000*"article_NOUN" + 0.000*"object_NOUN"
topic №5 :   kwds  0.000*"language_NOUN" + 0.000*"type_NOUN" + 0.000*"article_NOUN" + 0.000*"description_NOUN" + 0.000*"image_NOUN"
topic №6 :   kwds  0.024*"speech_NOUN" + 0.020*"word_NOUN" + 0.012*"Estonian_ADJ" + 0.010*"duration_NOUN" + 0.009*"phonological_ADJ"
topic №7 :   kwds  0.023*"language_NOUN" + 0.023*"type_NOUN" + 0.020*"image_NOUN" + 0.018*

log = -9.14278396016
topic №0 :   kwds  0.008*"language_NOUN" + 0.007*"dictionary_NOUN" + 0.006*"fifth_ADJ" + 0.006*"analysis_NOUN" + 0.005*"text_NOUN"
topic №1 :   kwds  0.017*"term_NOUN" + 0.012*"text_NOUN" + 0.007*"language_NOUN" + 0.007*"relation_NOUN" + 0.005*"report_NOUN"
topic №2 :   kwds  0.009*"semantic_ADJ" + 0.008*"error_NOUN" + 0.006*"language_NOUN" + 0.006*"construction_NOUN" + 0.005*"service_NOUN"
topic №3 :   kwds  0.010*"system_NOUN" + 0.009*"service_NOUN" + 0.006*"patient_NOUN" + 0.005*"shall_AUX" + 0.005*"action_NOUN"
topic №4 :   kwds  0.011*"text_NOUN" + 0.009*"relation_NOUN" + 0.009*"disease_NOUN" + 0.008*"concept_NOUN" + 0.007*"semantic_ADJ"
topic №5 :   kwds  0.009*"language_NOUN" + 0.006*"algorithm_NOUN" + 0.005*"syntactic_ADJ" + 0.004*"motion_NOUN" + 0.004*"robot_NOUN"
topic №6 :   kwds  0.005*"energy_NOUN" + 0.003*"system_NOUN" + 0.003*"operator_NOUN" + 0.003*"rhythm_NOUN" + 0.003*"peak_NOUN"
topic №7 :   kwds  0.010*"model_NOUN" + 0.009*"similarity_NOUN" + 0.

#### По всем годам без частей речи, topics = 10, removed 20 most freq_words

In [20]:
frq = 20
topics = 10
raw = 1
if raw == 1:
    data = all_texts_dict_raw
elif raw == 0:
    data = all_texts_dict_pos
else:
    raise Exception
old_data = data
for y in years:
    data = _get_data_distribution(old_data, "years/"+str(y)+"_en.pickle")
    ldamodel = gensim.models.ldamodel.LdaModel.load("new_output/years/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+"/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(y)+"_en.mdl")
    print("/lda_model_normalized_words_freq_"+str(y)+"_en"+str(frq)+"_topics_"+str(topics)+"_pos_"+str(raw)+str(y)+"_en.mdl")
    top_topics = {}
    dictionary = corpora.Dictionary(data)
    dictionary.filter_n_most_frequent(frq)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    print("log =",ldamodel.log_perplexity(corpus))
    all_topics = dict(ldamodel.print_topics(num_topics=topics, num_words=5))
    for a in all_topics:
        print("topic №"+str(a),": ",  " kwds ", all_topics[a])
    print()
    print()
    print()

    print("*"*20)

/lda_model_normalized_words_freq_2001_en20_topics_10_pos_12001_en.mdl
log = -6.56002361808
topic №0 :   kwds  0.001*"answer" + 0.001*"recognition" + 0.001*"tone" + 0.001*"give" + 0.001*"question"
topic №1 :   kwds  0.001*"answer" + 0.001*"give" + 0.001*"context" + 0.001*"possible" + 0.001*"ground"
topic №2 :   kwds  0.001*"recognition" + 0.001*"answer" + 0.001*"verification" + 0.001*"datum" + 0.001*"version"
topic №3 :   kwds  0.001*"answer" + 0.001*"recognition" + 0.001*"question" + 0.001*"possible" + 0.001*"verification"
topic №4 :   kwds  0.029*"consonant" + 0.029*"speech" + 0.023*"diphone" + 0.019*"error" + 0.019*"Estonian"
topic №5 :   kwds  0.001*"speech" + 0.001*"consonant" + 0.001*"recognition" + 0.001*"error" + 0.001*"Estonian"
topic №6 :   kwds  0.044*"recognition" + 0.025*"version" + 0.025*"tone" + 0.025*"verification" + 0.023*"datum"
topic №7 :   kwds  0.001*"recognition" + 0.001*"verification" + 0.001*"version" + 0.001*"tone" + 0.001*"datum"
topic №8 :   kwds  0.001*"recog

log = -7.9971329352
topic №0 :   kwds  0.031*"event" + 0.018*"case" + 0.018*"ller" + 0.016*"role" + 0.013*"attribute"
topic №1 :   kwds  0.001*"language" + 0.001*"object" + 0.001*"type" + 0.001*"Estonian" + 0.001*"word"
topic №2 :   kwds  0.001*"event" + 0.001*"Abfall" + 0.001*"role" + 0.001*"case" + 0.001*"ller"
topic №3 :   kwds  0.001*"object" + 0.001*"event" + 0.001*"image" + 0.001*"word" + 0.001*"type"
topic №4 :   kwds  0.001*"language" + 0.001*"object" + 0.001*"article" + 0.001*"image" + 0.001*"type"
topic №5 :   kwds  0.037*"dictionary" + 0.022*"entry" + 0.020*"system" + 0.013*"language" + 0.012*"Estonian"
topic №6 :   kwds  0.024*"Abfall" + 0.017*"meaning" + 0.013*"Mullvermeidung" + 0.011*"Mull" + 0.009*"context"
topic №7 :   kwds  0.028*"object" + 0.025*"type" + 0.024*"language" + 0.024*"image" + 0.020*"article"
topic №8 :   kwds  0.001*"event" + 0.001*"case" + 0.001*"dictionary" + 0.001*"role" + 0.001*"entry"
topic №9 :   kwds  0.030*"word" + 0.025*"speech" + 0.021*"Estonian

log = -7.9649392277
topic №0 :   kwds  0.010*"algorithm" + 0.008*"adjective" + 0.007*"topic" + 0.007*"cluster" + 0.006*"role"
topic №1 :   kwds  0.027*"relation" + 0.016*"expression" + 0.012*"concept" + 0.012*"time" + 0.011*"part"
topic №2 :   kwds  0.009*"text" + 0.008*"Russian" + 0.007*"dataset" + 0.007*"language" + 0.006*"corpus"
topic №3 :   kwds  0.017*"entity" + 0.015*"name" + 0.014*"text" + 0.013*"fact" + 0.011*"system"
topic №4 :   kwds  0.015*"system" + 0.014*"model" + 0.013*"feature" + 0.012*"language" + 0.010*"speech"
topic №5 :   kwds  0.017*"user" + 0.013*"model" + 0.006*"language" + 0.005*"network" + 0.005*"social"
topic №6 :   kwds  0.012*"Russian" + 0.011*"model" + 0.010*"language" + 0.007*"sentence" + 0.007*"text"
topic №7 :   kwds  0.027*"sentiment" + 0.017*"lexicon" + 0.013*"dictionary" + 0.010*"positive" + 0.009*"aspect"
topic №8 :   kwds  0.021*"term" + 0.011*"analyzer" + 0.010*"TOROT" + 0.008*"lemma" + 0.007*"guess"
topic №9 :   kwds  0.024*"language" + 0.014*"Rus

Я думаю, что если захочется запустить что-то, но с другими параметрами, то ты сможешь поменять циферки